In [27]:
cd ..

/home/soda/rcappuzz/work/benchmark-join-suggestions


In [28]:
import polars as pl

In [29]:
df = pl.DataFrame(
    {
        "id": [0, 0, 0, 1, 1],
        "color": ["red", "green", "green", "red", "red"],
        "shape": ["square", "triangle", "square", "triangle", "square"],
    }
)


In [30]:
df_list = []
for gkey, group in df.groupby("id"):
    g = group.select(pl.col("id"),
       pl.all().exclude("id").mode().first()
    ).unique()
    df_list.append(g)

df_dedup = pl.concat(df_list)

In [31]:
df_dedup

id,color,shape
i64,str,str
1,"""red""","""triangle"""
0,"""green""","""square"""


In [32]:
import pickle
with open(f"generated_candidates_movies-prepared.pickle", "rb") as fp:
    candidates = pickle.load(fp)


In [34]:
c_mh = candidates["minhash"]

In [49]:
cand_join = c_mh.popitem()[1]

In [45]:
src_md = cand_join["source_metadata"]

{'indexing_method': 'minhash',
 'source_table': '310b7e82655f2beaba69e8b482d43b65',
 'candidate_table': '493060039c2bd6f510fead93e2086f3744166236cccc9c787e768b8787e79024',
 'source_metadata': {'full_path': '/storage/store/work/rcappuzz/ken_datasets/the-movies-dataset/movies-prepared.parquet',
  'hash': '310b7e82655f2beaba69e8b482d43b65',
  'df_name': 'movies-prepared',
  'source_dl': 'queries',
  'license': '',
  'path_metadata': '/storage/store/work/rcappuzz/metadata/queries/310b7e82655f2beaba69e8b482d43b65.json'},
 'candidate_metadata': {'full_path': '/storage/store/work/rcappuzz/yago3-dl/wordnet/yago_wordnet_person.parquet',
  'hash': '493060039c2bd6f510fead93e2086f3744166236cccc9c787e768b8787e79024',
  'df_name': 'yago_wordnet_person',
  'source_dl': 'yago3-dl',
  'license': '',
  'path_metadata': '/storage/store/work/rcappuzz/metadata/wordnet/493060039c2bd6f510fead93e2086f3744166236cccc9c787e768b8787e79024.json'},
 'similarity_score': 20,
 'how': 'left',
 'left_on': ['col_to_embed

In [54]:
src_md = cand_join.source_metadata
cand_md = cand_join.candidate_metadata

In [57]:
src_table = pl.read_parquet(src_md["full_path"])
cand_table = pl.read_parquet(cand_md["full_path"])

In [59]:
from src.table_integration.utils_joins import execute_join

In [64]:
src_table.unique()

adult,budget,genres,id,original_language,original_title,popularity,production_companies,production_countries,release_date,runtime,spoken_languages,status,title,video,vote_average,vote_count,col_to_embed,target
str,str,str,str,str,str,str,str,str,str,f64,str,str,str,bool,f64,f64,str,f64
"""False""","""44000000""","""History""","""10858""","""en""","""Nixon""","""5.092""","""Hollywood Pict…","""US""","""1995""",192.0,"""English""","""Released""","""Nixon""",false,7.1,72.0,"""<Nixon_(film)>…",7.136142
"""False""","""0""","""Drama""","""18215""","""en""","""Boxing Helena""","""4.304224""","""Mainline Pictu…","""US""","""1993""",107.0,"""English""","""Released""","""Boxing Helena""",false,4.5,59.0,"""<Boxing_Helena…",6.2544
"""False""","""0""","""Comedy""","""24405""","""en""","""Chasers""","""9.85327""","""Warner Bros.""","""US""","""1994""",102.0,"""English""","""Released""","""Chasers""",false,4.7,22.0,"""<Chasers>""",6.20322
"""False""","""4800000""","""Action""","""40507""","""en""","""Original Gangs…","""0.343579""","""Po' Boy Produc…","""US""","""1996""",99.0,"""Español""","""Released""","""Original Gangs…",false,5.5,5.0,"""<Original_Gang…",6.479143
"""False""","""1500000""","""Drama""","""4174""","""en""","""Spellbound""","""5.47596""","""Selznick Inter…","""US""","""1945""",111.0,"""English""","""Released""","""Spellbound""",false,7.3,166.0,"""<Spellbound_(1…",6.845098
"""False""","""1365000""","""Comedy""","""1859""","""en""","""Ninotchka""","""5.120036""","""Metro-Goldwyn-…","""US""","""1939""",110.0,"""English""","""Released""","""Ninotchka""",false,7.3,77.0,"""<Ninotchka>""",6.357744
"""False""","""8000000""","""Science Fictio…","""149""","""ja""","""アキラ""","""10.888917""","""Bandai Visual …","""JP""","""1988""",124.0,"""日本語""","""Released""","""Akira""",false,7.8,792.0,"""<Akira_(1988_f…",5.742859
"""False""","""16000000""","""Adventure""","""8009""","""en""","""Highlander""","""16.870392""","""Davis-Panzer P…","""GB""","""1986""",116.0,"""English""","""Released""","""Highlander""",false,6.8,642.0,"""<Highlander_(f…",6.770852
"""False""","""13200000""","""Comedy""","""9037""","""en""","""Grease 2""","""14.202364""","""Paramount Pict…","""US""","""1982""",115.0,"""English""","""Released""","""Grease 2""",false,4.9,185.0,"""<Grease_2>""",7.181028


In [62]:
execute_join(src_table, cand_table, 
             left_on=cand_join.left_on,
             right_on=cand_join.right_on,
             dedup=True
             )

adult,budget,genres,id,original_language,original_title,popularity,production_companies,production_countries,release_date,runtime,spoken_languages,status,title,video,vote_average,vote_count,col_to_embed,target,type,happenedOnDate,hasWebsite,hasWeight,happenedIn,wasDestroyedOnDate,hasDuration,isLocatedIn,hasLatitude,hasLongitude,wasCreatedOnDate,hasPages,hasISBN
str,str,str,str,str,str,str,str,str,str,f64,str,str,str,bool,f64,f64,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str
"""False""","""60000000""","""Action""","""949""","""en""","""Heat""","""17.924927""","""Regency Enterp…","""US""","""1995""",170.0,"""English""","""Released""","""Heat""",false,7.7,1886.0,"""<Heat_(1995_fi…",8.272855,"""<wordnet_movie…",null,null,null,null,null,null,"""<United_States…",null,null,"""1995-12-15^^xs…",null,null
"""False""","""44000000""","""History""","""10858""","""en""","""Nixon""","""5.092""","""Hollywood Pict…","""US""","""1995""",192.0,"""English""","""Released""","""Nixon""",false,7.1,72.0,"""<Nixon_(film)>…",7.136142,"""<wordnet_movie…",null,null,null,null,null,null,"""<United_States…",null,null,"""1995-12-22^^xs…",null,null
"""False""","""100000000""","""Action""","""414""","""en""","""Batman Forever…","""13.321354""","""Warner Bros.""","""GB""","""1995""",121.0,"""English""","""Released""","""Batman Forever…",false,5.2,1529.0,"""<Batman_Foreve…",8.527023,"""<wordnet_movie…",null,null,null,null,null,null,"""<United_States…",null,null,"""1995-06-09^^xs…",null,null
"""False""","""15000000""","""Drama""","""27303""","""en""","""Hideaway""","""8.182165""","""TriStar Pictur…","""US""","""1995""",103.0,"""English""","""Released""","""Hideaway""",false,5.0,24.0,"""<Hideaway_(199…",7.086405,"""<wordnet_movie…",null,null,null,null,null,null,"""<United_States…",null,null,"""1995-03-03^^xs…",null,null
"""False""","""0""","""Drama""","""29973""","""en""","""Cobb""","""2.583363""","""Regency Enterp…","""US""","""1994""",128.0,"""English""","""Released""","""Cobb""",false,6.1,29.0,"""<Cobb_(film)>""",6.003281,"""<wordnet_movie…",null,null,null,null,null,null,"""<United_States…",null,null,"""1994-12-02^^xs…",null,null
"""False""","""0""","""Action""","""11853""","""en""","""Bad Girls""","""14.644772""","""Twentieth Cent…","""US""","""1994""",99.0,"""English""","""Released""","""Bad Girls""",false,4.6,61.0,"""<Bad_Girls_(19…",7.182997,"""<wordnet_movie…",null,null,null,null,null,null,"""<United_States…",null,null,"""1994-04-22^^xs…",null,null
"""False""","""9000000""","""Thriller""","""10909""","""en""","""Kalifornia""","""8.4337""","""Propaganda Fil…","""US""","""1993""",117.0,"""English""","""Released""","""Kalifornia""",false,6.5,209.0,"""<Kalifornia>""",6.379347,"""<wordnet_movie…",null,null,null,null,null,null,"""<United_States…",null,null,"""1993-08-27^^xs…",null,null
"""False""","""2600000""","""Animation""","""10895""","""en""","""Pinocchio""","""14.202284""","""RKO Radio Pict…","""US""","""1940""",88.0,"""English""","""Released""","""Pinocchio""",false,6.9,1412.0,"""<Pinocchio>""",7.925828,null,null,null,null,null,null,null,null,null,null,null,null,null
"""False""","""14000000""","""Romance""","""114""","""en""","""Pretty Woman""","""13.348451""","""Touchstone Pic…","""US""","""1990""",119.0,"""English""","""Released""","""Pretty Woman""",false,7.0,1807.0,"""<Pretty_Woman>…",8.665581,"""<wordnet_movie…",null,null,null,null,null,null,"""<United_States…",null,null,"""1990-03-23^^xs…",null,null
